In [7]:
#トラカンデータの距離案分()
import numpy as np
import psycopg2
import datetime
import pandas as pd

In [207]:
#計測地点のosm_idを指定
start_osm = 307601946
mid_osm = [204689095,205060576]
end_osm =  47818676
start_mesh = 50326612413
end_mesh = 50326613332

In [208]:
connection = psycopg2.connectconnection = psycopg2.connect(host="localhost", database="analysisdb", user="postgres",password="setouchi1311")

In [209]:
getdbname = "kddi_18_new_param"
dbname = "kddi_trkn18_monthly_anbun"
trknName = "trkn_18_grp"
sttime = "2022-04-01 18:00:00"

In [210]:
for i in range(30):
    dte = datetime.datetime.strptime(sttime, '%Y-%m-%d %H:%M:%S')
    if i != 0:
        dte = dte + datetime.timedelta(days=i)
    for j in range(24):
        timestr = dte.strftime('%H:%M:%S')
        #特定日付のKDDIメッシュ情報を全取得
        cur =connection.cursor()
        sql = "SELECT mesh, tbran,pointdata.latitude,pointdata.longitude FROM " + getdbname + " "
        sql += "INNER JOIN pointdata ON mesh = cast(pointdata.grid_code AS BIGINT)  "
        sql += "WHERE times = '"+str(dte)+"' AND pointdata.grid_code != '0'  ORDER BY mesh "
        cur.execute(sql)
        kddiDf = pd.DataFrame.from_records(cur.fetchall(), columns=["mesh","tbran","latitude","longitude"])
        cur.close()

        one_target_list = []
        one_target_list.append(start_mesh)
        one_target_list.append(end_mesh)
        two_target_list = []
        two_target_list.append(start_osm)
        if len(mid_osm) != 0:
            for target in mid_osm:
                two_target_list.append(target)
        if start_osm != end_osm:
            two_target_list.append(end_osm)

        cur =connection.cursor()
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(start_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            stlat = row[0]
            stlng = row[1]
        sql = "SELECT latitude,longitude FROM pointdata WHERE pointdata.grid_code = '"+str(end_mesh)+"'"
        cur.execute(sql)
        for row in cur:
            edlat = row[0]
            edlng = row[1]
        cur.close()

        if stlat > edlat:
            wklat = stlat
            stlat = edlat
            edlat = wklat
        if stlng > edlng:
            wklng = stlng
            stlng = edlng
            edlng = wklng

        print(str(stlat) + ":" + str(edlat) + ":" + str(stlng) + ":" + str(stlng))

        keylist = []
        cnt = 0
        for item in two_target_list:
            sql = "SELECT a.grid_code,a.latitude,a.longitude"
            sql += " FROM pointdata a, roadmap r"
            sql += " WHERE osm_id = "+ str(item) +" and ST_Distance(a.geom, r.geom) < 0.001"
            sql += " order by ST_Distance(a.geom, r.geom);"
            cur =connection.cursor()
            cur.execute(sql)
            childlist = []
            for row in cur:
                if row[1] >= stlat and row[1] <= edlat and row[2] >= stlng and row[2] <= edlng:
                    childlist.append([row[0], row[1], row[2]])
                #print(row[0])
            worklist = [item, childlist]
            keylist.append(worklist)
            cnt = cnt + 1
            cur.close()
        #特定日付のトラカンデータを全取得 > 緯度経度
        cur =connection.cursor()
        #sql = "SELECT trakanmap.gridcode , 断面交通量,pointdata.latitude,pointdata.longitude FROM trakan2204 "
        #sql += "INNER JOIN trakanmap ON trakan2204.計測地点番号 = trakanmap.地点番号 "
        #sql += "INNER JOIN pointdata ON trakanmap.gridcode = pointdata.grid_code "
        #sql += "WHERE 時刻 = '"+str(dte)+"' AND (trakanmap.gridcode = '"+str(start_mesh)+"' or trakanmap.gridcode = '"+str(end_mesh)+"') "
        #sql += "ORDER BY trakanmap.gridcode"
        sql = "SELECT grid_code, tbran, latitude, longitude FROM " + trknName  + " "
        sql += "INNER JOIN pointdata ON mesh = cast(grid_code as bigint) "
        sql += "WHERE times = '"+timestr+"' AND (mesh = '"+str(start_mesh)+"' or mesh = '"+str(end_mesh)+"') "
        sql += "ORDER BY mesh"
        cur.execute(sql)
        trknDf = pd.DataFrame(cur.fetchall(), columns=["gridcode","danmen","latitude","longitude"])
        cur.close()
        #trknDf
        #keylist

        from geopy.distance import geodesic

        #データが存在する道路IDごとに交通量を平均化して取得する
        lstItems = []
        cntDt = 0
        wks = ""
        wknp=""
        find = 0

        trknStParam = trknDf.query('gridcode == "'+str(start_mesh) + '"')
        trknEdParam = trknDf.query('gridcode == "'+str(end_mesh) + '"')

        if len(trknStParam) == 0:
            print(str(start_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknStParam) == 1:
            trknSt = trknStParam['danmen'].iloc[-1]
        else:
            trknSt = trknStParam['danmen'].iloc[0] + trknStParam['danmen'].iloc[1]

        if len(trknEdParam) == 0:
            print(str(end_mesh) + " mesh empty!")
            dte = dte + datetime.timedelta(minutes=15) 
            continue
        elif len(trknEdParam) == 1:
            trknEd = trknEdParam['danmen'].iloc[-1]
        else:
            trknEd = trknEdParam['danmen'].iloc[0] + trknEdParam['danmen'].iloc[1]

        st_station = [trknStParam['latitude'].iloc[-1], trknStParam['longitude'].iloc[-1]]
        ed_station = [trknEdParam['latitude'].iloc[-1], trknEdParam['longitude'].iloc[-1]]

        dis_km = geodesic(st_station, ed_station)
        cur =connection.cursor()
        for row in keylist:
            for meshid in row[1]:
                #路線のメッシュ数分ループ
                print(str(meshid[0]))
                kddiParam = kddiDf.query('mesh == '+str(meshid[0]))
                if len(kddiParam) == 0:
                    print(str(meshid[0]) + " is kddi none!")
                    continue
                #路線のメッシュがkddiに存在したなら
                
                kddi_station = [kddiParam['latitude'].iloc[-1], kddiParam['longitude'].iloc[-1]]
                kddi_km = geodesic(st_station, kddi_station)
                
                if trknSt < trknEd:
                    ret = ((trknEd - trknSt) * (kddi_km / dis_km)) + trknSt
                else:
                    ret = trknSt - ((trknEd - trknSt) * (kddi_km / dis_km))
                print("存在メッシュ：" + str(meshid[0]) + "値" + str(ret) + "距離" + str(kddi_km))
                sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(meshid[0]) + ","  +  str(ret) + ") ON CONFLICT DO NOTHING;"
                cur.execute(sql)
                connection.commit()

        sql = "INSERT INTO "+dbname+" (times, stmesh, edmesh, mesh, tbran) VALUES ('"+str(dte)+"',"+str(start_mesh)+","  +str(end_mesh)+","  + str(start_mesh) + ","  + str(trknSt) + ") ON CONFLICT DO NOTHING;"
        cur.execute(sql)
        connection.commit()
        cur.close()
        dte = dte + datetime.timedelta(minutes=15) 
        #print(lstItems)


33.8473958335:33.8484375000:132.782031250:132.782031250
50326612413
存在メッシュ：50326612413値52.68333333333333距離0.0 km
50326612414
存在メッシュ：50326612414値59.621668204402376距離0.14460884879856797 km
50326612432
存在メッシュ：50326612432値61.564335998337015距離0.18509795151546485 km
50326612441
存在メッシュ：50326612441値67.62628912979322距離0.31144124282387203 km
50326612414
存在メッシュ：50326612414値59.621668204402376距離0.14460884879856797 km
50326612423
存在メッシュ：50326612423値66.56000307507495距離0.2892176975888725 km
50326612432
存在メッシュ：50326612432値61.564335998337015距離0.18509795151546485 km
50326613332
存在メッシュ：50326613332値87.81494252873563距離0.7322133705843374 km
50326613314
存在メッシュ：50326613314値87.37500768054531距離0.7230442438233267 km
50326613331
存在メッシュ：50326613331値80.98476096811788距離0.5898586542265849 km
33.8473958335:33.8484375000:132.782031250:132.782031250
50326612413
存在メッシュ：50326612413値50.13333333333333距離0.0 km
50326612414
存在メッシュ：50326612414値56.571686769943135距離0.14460884879856797 km
50326612432
存在メッシュ：50326612432値58.374364516

33.8473958335:33.8484375000:132.782031250:132.782031250
50326612413
存在メッシュ：50326612413値26.266666666666666距離0.0 km
50326612414
存在メッシュ：50326612414値29.834752109041425距離0.14460884879856797 km
50326612432
存在メッシュ：50326612432値30.833782107407654距離0.18509795151546485 km
50326612441
存在メッシュ：50326612441値33.95118233709336距離0.31144124282387203 km
50326612414
存在メッシュ：50326612414値29.834752109041425距離0.14460884879856797 km
50326612423
存在メッシュ：50326612423値33.402837551212286距離0.2892176975888725 km
50326612432
存在メッシュ：50326612432値30.833782107407654距離0.18509795151546485 km
50326613332
存在メッシュ：50326613332値44.33333333333333距離0.7322133705843374 km
50326613314
存在メッシュ：50326613314値44.10709387435787距離0.7230442438233267 km
50326613331
存在メッシュ：50326613331値40.820866459352416距離0.5898586542265849 km
33.8473958335:33.8484375000:132.782031250:132.782031250
50326612413
存在メッシュ：50326612413値21.866666666666667距離0.0 km
50326612414
存在メッシュ：50326612414値24.713893518746154距離0.14460884879856797 km
50326612432
存在メッシュ：50326612432値25.51108

33.8473958335:33.8484375000:132.782031250:132.782031250
50326612413
存在メッシュ：50326612413値7.966666666666667距離0.0 km
50326612414
存在メッシュ：50326612414値9.105557407498463距離0.14460884879856797 km
50326612432
存在メッシュ：50326612432値9.424435986312776距離0.18509795151546485 km
50326612441
存在メッシュ：50326612441値10.419473328998434距離0.31144124282387203 km
50326612414
存在メッシュ：50326612414値9.105557407498463距離0.14460884879856797 km
50326612423
存在メッシュ：50326612423値10.244448148265178距離0.2892176975888725 km
50326612432
存在メッシュ：50326612432値9.424435986312776距離0.18509795151546485 km
50326613332
存在メッシュ：50326613332値13.733333333333334距離0.7322133705843374 km
50326613314
存在メッシュ：50326613314値13.661120369490614距離0.7230442438233267 km
50326613331
存在メッシュ：50326613331値12.612195382782229距離0.5898586542265849 km
33.8473958335:33.8484375000:132.782031250:132.782031250
50326612413
存在メッシュ：50326612413値9.03448275862069距離0.0 km
50326612414
存在メッシュ：50326612414値10.466552170328361距離0.14460884879856797 km
50326612432
存在メッシュ：50326612432値10.867517989

33.8473958335:33.8484375000:132.782031250:132.782031250
50326612413
存在メッシュ：50326612413値26.666666666666668距離0.0 km
50326612414
存在メッシュ：50326612414値30.116254806527134距離0.14460884879856797 km
50326612432
存在メッシュ：50326612432値31.082106686866446距離0.18509795151546485 km
50326612441
存在メッシュ：50326612441値34.095977019625316距離0.31144124282387203 km
50326612414
存在メッシュ：50326612414値30.116254806527134距離0.14460884879856797 km
50326612423
存在メッシュ：50326612423値33.56584294619048距離0.2892176975888725 km
50326612432
存在メッシュ：50326612432値31.082106686866446距離0.18509795151546485 km
50326613332
存在メッシュ：50326613332値44.13333333333334距離0.7322133705843374 km
50326613314
存在メッシュ：50326613314値43.914607361925334距離0.7230442438233267 km
50326613331
存在メッシュ：50326613331値40.737516650739245距離0.5898586542265849 km
33.8473958335:33.8484375000:132.782031250:132.782031250
50326612413
存在メッシュ：50326612413値25.066666666666666距離0.0 km
50326612414
存在メッシュ：50326612414値28.43396501311443距離0.14460884879856797 km
50326612432
存在メッシュ：50326612432値29.37677

33.8473958335:33.8484375000:132.782031250:132.782031250
50326612413
存在メッシュ：50326612413値11.6距離0.0 km
50326612414
存在メッシュ：50326612414値13.249087459990548距離0.14460884879856797 km
50326612432
存在メッシュ：50326612432値13.710816269198556距離0.18509795151546485 km
50326612441
存在メッシュ：50326612441値15.151607334763627距離0.31144124282387203 km
50326612414
存在メッシュ：50326612414値13.249087459990548距離0.14460884879856797 km
50326612423
存在メッシュ：50326612423値14.898174919886861距離0.2892176975888725 km
50326612432
存在メッシュ：50326612432値13.710816269198556距離0.18509795151546485 km
50326613332
存在メッシュ：50326613332値19.95距離0.7322133705843374 km
50326613314
存在メッシュ：50326613314値19.845437298019647距離0.7230442438233267 km
50326613331
存在メッシュ：50326613331値18.326618169866755距離0.5898586542265849 km
33.8473958335:33.8484375000:132.782031250:132.782031250
50326612413
存在メッシュ：50326612413値10.516666666666667距離0.0 km
50326612414
存在メッシュ：50326612414値12.043965232406416距離0.14460884879856797 km
50326612432
存在メッシュ：50326612432値12.471594309197865距離0.1850979515

...

33.8473958335:33.8484375000:132.782031250:132.782031250
50326612413
存在メッシュ：50326612413値12.033333333333333距離0.0 km
50326612414
存在メッシュ：50326612414値13.70217034374293距離0.14460884879856797 km
50326612432
存在メッシュ：50326612432値14.16942883928876距離0.18509795151546485 km
50326612441
存在メッシュ：50326612441値15.627474887674968距離0.31144124282387203 km
50326612414
存在メッシュ：50326612414値13.70217034374293距離0.14460884879856797 km
50326612423
存在メッシュ：50326612423値15.371007354057163距離0.2892176975888725 km
50326612432
存在メッシュ：50326612432値14.16942883928876距離0.18509795151546485 km
50326613332
存在メッシュ：50326613332値20.483333333333334距離0.7322133705843374 km
50326613314
存在メッシュ：50326613314値20.37751838342507距離0.7230442438233267 km
50326613331
存在メッシュ：50326613331値18.840509804635616距離0.5898586542265849 km
33.8473958335:33.8484375000:132.782031250:132.782031250
50326612413
存在メッシュ：50326612413値11.6距離0.0 km
50326612414
存在メッシュ：50326612414値13.249087459990548距離0.14460884879856797 km
50326612432
存在メッシュ：50326612432値13.710816269198556距離0.18

33.8473958335:33.8484375000:132.782031250:132.782031250
50326612413
存在メッシュ：50326612413値37.7距離0.0 km
50326612414
存在メッシュ：50326612414値42.66372033865419距離0.14460884879856797 km
50326612432
存在メッシュ：50326612432値44.0535148382264距離0.18509795151546485 km
50326612441
存在メッシュ：50326612441値48.39026718727256距離0.31144124282387203 km
50326612414
存在メッシュ：50326612414値42.66372033865419距離0.14460884879856797 km
50326612423
存在メッシュ：50326612423値47.62744067702472距離0.2892176975888725 km
50326612432
存在メッシュ：50326612432値44.0535148382264距離0.18509795151546485 km
50326613332
存在メッシュ：50326613332値62.83333333333334距離0.7322133705843374 km
50326613314
存在メッシュ：50326613314値62.51860168745236距離0.7230442438233267 km
50326613331
存在メッシュ：50326613331値57.946986427463216距離0.5898586542265849 km
33.8473958335:33.8484375000:132.782031250:132.782031250
50326612413
存在メッシュ：50326612413値32.56666666666667距離0.0 km
50326612414
存在メッシュ：50326612414値36.86548547454622距離0.14460884879856797 km
50326612432
存在メッシュ：50326612432値38.06911386741181距離0.1850979515

33.8473958335:33.8484375000:132.782031250:132.782031250
50326612413
存在メッシュ：50326612413値14.8距離0.0 km
50326612414
存在メッシュ：50326612414値16.830912101425486距離0.14460884879856797 km
50326612432
存在メッシュ：50326612432値17.399548179831356距離0.18509795151546485 km
50326612441
存在メッシュ：50326612441値19.17393557993844距離0.31144124282387203 km
50326612414
存在メッシュ：50326612414値16.830912101425486距離0.14460884879856797 km
50326612423
存在メッシュ：50326612423値18.86182420273492距離0.2892176975888725 km
50326612432
存在メッシュ：50326612432値17.399548179831356距離0.18509795151546485 km
50326613332
存在メッシュ：50326613332値25.083333333333336距離0.7322133705843374 km
50326613314
存在メッシュ：50326613314値24.954560504746752距離0.7230442438233267 km
50326613331
存在メッシュ：50326613331値23.084078664287006距離0.5898586542265849 km
33.8473958335:33.8484375000:132.782031250:132.782031250
50326612413
存在メッシュ：50326612413値13.266666666666667距離0.0 km
50326612414
存在メッシュ：50326612414値15.086916896955636距離0.14460884879856797 km
50326612432
存在メッシュ：50326612432値15.596569654424755距離0

33.8473958335:33.8484375000:132.782031250:132.782031250
50326612413
存在メッシュ：50326612413値40.016666666666666距離0.0 km
50326612414
存在メッシュ：50326612414値45.312837770708164距離0.14460884879856797 km
50326612432
存在メッシュ：50326612432値46.79571532363369距離0.18509795151546485 km
50326612441
存在メッシュ：50326612441値51.42292655016902距離0.31144124282387203 km
50326612414
存在メッシュ：50326612414値45.312837770708164距離0.14460884879856797 km
50326612423
存在メッシュ：50326612423値50.60900887444702距離0.2892176975888725 km
50326612432
存在メッシュ：50326612432値46.79571532363369距離0.18509795151546485 km
50326613332
存在メッシュ：50326613332値66.83333333333333距離0.7322133705843374 km
50326613314
存在メッシュ：50326613314値66.4975221806659距離0.7230442438233267 km
50326613331
存在メッシュ：50326613331値61.61971783496129距離0.5898586542265849 km
33.8473958335:33.8484375000:132.782031250:132.782031250
50326612413
存在メッシュ：50326612413値39.55距離0.0 km
50326612414
存在メッシュ：50326612414値44.52688670560021距離0.14460884879856797 km
50326612432
存在メッシュ：50326612432値45.92036766273097距離0.185097

33.8473958335:33.8484375000:132.782031250:132.782031250
50326612413
存在メッシュ：50326612413値19.166666666666668距離0.0 km
50326612414
存在メッシュ：50326612414値21.707775487250906距離0.14460884879856797 km
50326612432
存在メッシュ：50326612432値22.41926179605047距離0.18509795151546485 km
50326612441
存在メッシュ：50326612441値24.639402919036968距離0.31144124282387203 km
50326612414
存在メッシュ：50326612414値21.707775487250906距離0.14460884879856797 km
50326612423
存在メッシュ：50326612423値24.248884307689934距離0.2892176975888725 km
50326612432
存在メッシュ：50326612432値22.41926179605047距離0.18509795151546485 km
50326613332
存在メッシュ：50326613332値32.03333333333333距離0.7322133705843374 km
50326613314
存在メッシュ：50326613314値31.87221076660912距離0.7230442438233267 km
50326613331
存在メッシュ：50326613331値29.531834784704863距離0.5898586542265849 km
33.8473958335:33.8484375000:132.782031250:132.782031250
50326612413
存在メッシュ：50326612413値16.3距離0.0 km
50326612414
存在メッシュ：50326612414値18.482325321304856距離0.14460884879856797 km
50326612432
存在メッシュ：50326612432値19.09335566163402距離0.18

In [211]:
connection.close()

In [171]:
#CreateGeojsonFile